# Image Augmentation
> Simple wrappers around TorchIO augmentations for usuage with fastai

In [1]:
import sys
sys.path.append('..')

In [2]:
# export
# default_exp augment
import torchio as tio
from fastai.vision.augment import RandTransform
from faimed3d.all import *

from libs.utils import get_dataframe

In [3]:
DATA_DIR = Path('/media/ScaleOut/vahldiek/MRI/SIJ')
df = get_dataframe(denoised=True)
image = TensorDicom3D.create(DATA_DIR/df.TIRM[0])

In [4]:
# export
class TorchIOTransform(RandTransform):
    def __init__(self, tfm_name, p=0.2, **kwargs):
        super().__init__(p=p)
        self.transform = getattr(tio, tfm_name)(**kwargs)
    
    def encodes(self, x:TensorMask3D):
        return x
    
    def encodes(self, x:TensorDicom3D):
        assert x.device == torch.device('cpu'), "No cuda support for torchIO transforms"
        meta = x._metadata
        if x.ndim == 3: x = x.unsqueeze(0)
        x = TensorDicom3D(self.transform(x)).squeeze()
        x._metadata = meta
        return x    

In [5]:
@patch
@delegates(BasicViewer)
def show(x:TensorDicom3D, **kwargs):
    BasicViewer(x, **kwargs).show()

In [6]:
tio_tfms_list = ['RandomBiasField', 'RandomGhosting', 'RandomMotion', 'RandomGamma', 'RandomAnisotropy', 'RandomElasticDeformation']

In [7]:
# export
def TioTransforms(p_all=None, 
                  tfms = ['RandomBiasField', 'RandomGhosting', 'RandomMotion', 
                          'RandomGamma', 'RandomAnisotropy']):
    if not p_all: p_all = 1/len(tfms)
    return [TorchIOTransform(tfm_name, p=p_all) for tfm_name in tfms]

In [8]:
images = [TorchIOTransform(tfm_name, p=1)(image, split_idx = 0) for tfm_name in tio_tfms_list]

/home/bressekk/Documents/GitHub/torchio/torchio/transforms/augmentation/spatial/random_elastic_deformation.py:244: RuntimeWarning: The maximum displacement is larger than the coarse grid spacing for dimensions: [0], so folding may occur. Choose fewer control points or a smaller maximum displacement
  warnings.warn(message, RuntimeWarning)


In [9]:
ListViewer(images, tio_tfms_list).show()

In [10]:
# export
class TioRandomBiasField(TorchIOTransform):
    def __init__(self, **kwargs):
        super().__init__(tfm_name = 'RandomBiasField', **kwargs)
        
class TioRandomGhosting(TorchIOTransform):
    def __init__(self, **kwargs):
        super().__init__(tfm_name = 'RandomGhosting', **kwargs)
               
class TioRandomMotion(TorchIOTransform):
    def __init__(self, **kwargs):
        super().__init__(tfm_name = 'RandomMotion', **kwargs)
        
class TioRandomGamma(TorchIOTransform):
    def __init__(self, **kwargs):
        super().__init__(tfm_name = 'RandomGamma', **kwargs)
        
class TioRandomAnisotropy(TorchIOTransform):
    def __init__(self, **kwargs):
        super().__init__(tfm_name = 'RandomAnisotropy', **kwargs)
        
class TioRandomElasticDeformation(TorchIOTransform):
    def __init__(self, **kwargs):
        super().__init__(tfm_name = 'RandomElasticDeformation', **kwargs)

In [ ]:
TioRandomElasticDeformation()

In [11]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_utils.ipynb.
Converted 01_preprocessing.ipynb.
Converted 02_augment.ipynb.
Converted 03_noise-reduction-network.ipynb.
Converted 04_3d_classification.ipynb.
Converted 05_evaluation.ipynb.
Converted 99_widgets.cam.ipynb.
Converted index.ipynb.
